In [1]:
import os
import operator
import cv2
import shutil
import random
import os
from shutil import copyfile


In [2]:
source_address = str(input("Enter the address of the source folder: "))
source_folder = str(input("Enter the name of the source folder where all the classes are stored: "))
test_address = str(input("Enter the address of the test images which will be used for testing after training: "))

Enter the address of the source folder: /home/rajarshi/Documents/DATASETS/Health/COVID/covid-chestxray-dataset-master/DATASET
Enter the name of the source folder where all the classes are stored: train
Enter the address of the test images which will be used for testing after training: /home/rajarshi/Documents/DATASETS/Health/COVID/covid-chestxray-dataset-master/DATASET/test


In [3]:
source_folder_dir = os.path.join(source_address, source_folder)

In [4]:
source_data = sum([len(files) for r, d, files in os.walk(source_folder_dir)])
print("Total no. of files: {}".format(source_data))

Total no. of files: 360


In [5]:
class_no = len(os.listdir(source_folder_dir))

classes = []
source_folder_dir_class = []
image_list = []
resized_images = []

files = {}
for i in range(0,class_no):
    i += 1
    classes = os.listdir(source_folder_dir)

    source_folder_dir_class.append(os.path.join(source_folder_dir, classes[i-1]))
    
    for filename in os.listdir(source_folder_dir_class[i-1]):
        file = os.path.join(source_folder_dir_class[i-1], filename)
        extension = "." + str(filename.split(".")[-1])
        #print("The format of the images: " + str(extension))
        x = os.path.getsize(file) 
        files.update({file:x})
    
    sorted_files = sorted(files.items(), key=operator.itemgetter(1))
    img_src = sorted_files[0][0]

    image = cv2.imread(img_src)
    #print(image.shape)
    height = image.shape[0]
    width = image.shape[1]

In [6]:
print("The size(height*width) of the smallest image in this dataset is {}*{}".format(height,width))

The size(height*width) of the smallest image in this dataset is 324*331


In [7]:
directory = os.path.join(source_address, 'classify ' + str(source_folder) )
train_directory = os.path.join(directory, 'training')
validation_directory = os.path.join(directory, 'validation')
test_directory = os.path.join(directory, 'testing')

# If the generated target folder is not created yet
if not os.path.exists(directory):
    # Create the target folder
    os.mkdir(directory)
# If the generated target folder is already created
else:
    # Overwrite the target folder
    shutil.rmtree(directory)
    os.mkdir(directory)

os.mkdir(train_directory)
os.mkdir(validation_directory)
os.mkdir(test_directory)

In [8]:
def split_data(SOURCE, TRAINING, VALIDATION, TESTING, VALIDATION_SPLIT_SIZE, TEST_SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = os.path.join(SOURCE, filename)
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + " is zero length, so ignoring!")

    # Splitting the Data    
    validation_length = int(len(files) * VALIDATION_SPLIT_SIZE)
    testing_length = int(len(files) * TEST_SPLIT_SIZE)
    training_length = len(files) - (validation_length + testing_length)
    
    shuffled_set = random.sample(files, len(files))
    
    training_set = shuffled_set[0:training_length]
    validation_set = shuffled_set[(training_length):(training_length+validation_length)]
    testing_set = shuffled_set[(training_length+validation_length):(training_length+validation_length+testing_length)]

    # Copying the Data
    for filename in training_set:
        this_file = os.path.join(SOURCE, filename)
        destination = os.path.join(TRAINING ,filename)
        copyfile(this_file, destination)
        
    for filename in validation_set:
        this_file = os.path.join(SOURCE, filename)
        destination = os.path.join(VALIDATION, filename)
        copyfile(this_file, destination)

    for filename in testing_set:
        this_file = os.path.join(SOURCE, filename)
        destination = os.path.join(TESTING, filename)
        copyfile(this_file, destination)

In [9]:
VALIDATION_SPLIT_SIZE = int(input("Using how much data(%) you want to validate? "))
VALIDATION_SPLIT_SIZE = VALIDATION_SPLIT_SIZE/100

TEST_SPLIT_SIZE = int(input("Using how much data(%) you want to test? "))
TEST_SPLIT_SIZE = TEST_SPLIT_SIZE/100

train_class = []
validation_class = []
test_class = []

for i in range(0,class_no):
    i += 1

    train_class.append(os.path.join(train_directory, classes[i-1]))
    validation_class.append(os.path.join(validation_directory, classes[i-1] ))
    test_class.append(os.path.join(test_directory, classes[i-1]))

    os.mkdir(train_class[i-1])
    os.mkdir(validation_class[i-1])
    os.mkdir(test_class[i-1])

    split_data(source_folder_dir_class[i-1], train_class[i-1], validation_class[i-1], test_class[i-1], VALIDATION_SPLIT_SIZE, TEST_SPLIT_SIZE)

    print("\nSplitted the RAW data from " + str(source_folder_dir_class[i-1]) + " & storing it into 3 folders at:\n" + str(train_class[i-1]) + '\n' + str(validation_class[i-1]) + '\n' + str(test_class[i-1]))


Using how much data(%) you want to validate? 10
Using how much data(%) you want to test? 10

Splitted the RAW data from /home/rajarshi/Documents/DATASETS/Health/COVID/covid-chestxray-dataset-master/DATASET/train/covid & storing it into 3 folders at:
/home/rajarshi/Documents/DATASETS/Health/COVID/covid-chestxray-dataset-master/DATASET/classify train/training/covid
/home/rajarshi/Documents/DATASETS/Health/COVID/covid-chestxray-dataset-master/DATASET/classify train/validation/covid
/home/rajarshi/Documents/DATASETS/Health/COVID/covid-chestxray-dataset-master/DATASET/classify train/testing/covid

Splitted the RAW data from /home/rajarshi/Documents/DATASETS/Health/COVID/covid-chestxray-dataset-master/DATASET/train/normal & storing it into 3 folders at:
/home/rajarshi/Documents/DATASETS/Health/COVID/covid-chestxray-dataset-master/DATASET/classify train/training/normal
/home/rajarshi/Documents/DATASETS/Health/COVID/covid-chestxray-dataset-master/DATASET/classify train/validation/normal
/home/